# Merging SOC 2018 and Census 2010 Occupational Codes



## Statement of the Problem

To explore the question of flexibility, the inherent properties of occupations should be considered. To do this, I use data from O\*Net to build an index of flexibility for a particular occupation. However, the O\*Net database uses the Standard Occupation Classification (SOC) Code, updated annually, while the CPS and ATUS use the 2010 Census Occupation Classification Codes. 

Linking the occupation codes requires utilizing the Crosswalk (https://www2.census.gov/programs-surveys/demo/guidance/industry-occupation/2018-occupation-code-list-and-crosswalk.xlsx). The design of the Crosswalk between 2018 SOC and 2010 Census is not friendly to easy linking. 

In order to use the Crosswalk to link the 2018 SOC with the 2010 Census, we must first extract some codes from the 2018 Census Title column, fill in empty cells (imported as NaN), create a dictionary, and append the 2010 Census codes to some O\*Net data. 

With the final .csv, we can merge the CPS/ATUS data with the O\*Net job characteristics via the 2010 Census occupation code. 

## Extracting 2018 SOC Codes

In [1]:
import pandas as pd

Download the crosswalk table, removing the first three rows of title, date updated, and white space. 

In [7]:
crosswalk = pd.read_csv('../2010 to 2018 Crosswalk -Table 1.csv',skiprows=3)

Looking at the first ten rows of the data, we see that there are SOC Codes in the Census Title column. We want to extract these codes and put them into the SOC Code column. 

In [8]:
crosswalk[0:10]

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers
2,11-1031,0030,Legislators,11-1031,30.0,Legislators
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN
5,NaN,NaN,NaN,11-2021,51.0,Marketing Managers
6,NaN,NaN,NaN,11-2022,52.0,Sales Managers
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers
8,NaN,NaN,NaN,NaN,NaN,Public Relations Managers (11-2032)
9,NaN,NaN,NaN,NaN,NaN,Fundraising Managers (11-2033)


To extract the codes, we are going to use the split function.

First we split along the first parenthesis.

In [9]:
crosswalk[['2018 Census Title short','tmp_Code']] = crosswalk['2018 Census Title '].str.split("(", expand=True)

In [10]:
crosswalk.head(10)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,tmp_Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,NaN,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,NaN,NaN,NaN,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032)
9,NaN,NaN,NaN,NaN,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033)


Now we split along the second parenthesis, and drop the unnecessary columns. 

In [11]:
crosswalk[['Code','paren']] = crosswalk['tmp_Code'].str.split(")", expand=True)
crosswalk.drop(['tmp_Code', 'paren'], axis = 1, inplace = True)

In [12]:
crosswalk.head(10)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,NaN,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,NaN,NaN,NaN,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032
9,NaN,NaN,NaN,NaN,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033


Now that we have extracted the SOC Codes from the Census title, we need to move the values to the 2018 SOC Code column

In [13]:
crosswalk['2018 SOC Code'].fillna(value=crosswalk['Code'], inplace=True)

In [14]:
crosswalk.head(10)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,NaN,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,NaN,NaN,11-2032,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032
9,NaN,NaN,NaN,11-2033,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033


Now, we want to forward fill the 2010 Census code to remove the NaN caused by blanks in the Excel spreadsheet.

In [15]:
crosswalk['2010 Census Code'].fillna(method = 'ffill', inplace = True)

In [16]:
crosswalk.head(15)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,0050,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,0050,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,0060,NaN,11-2032,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032
9,NaN,0060,NaN,11-2033,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033


## Dictionary to Match Census and SOC Codes

We are ready to create the dictionary. As we have many SOC codes to one Census code, we want to groupby SOC code and pull the relevant Census code.

In [17]:
crosswalk_dict = dict(crosswalk.groupby('2018 SOC Code')['2010 Census Code'].apply(list))

# crosswalk_dict

## Using Dictionary to Rename Occupation Codes

ONet to determine level of flexibility by occupation. Measures include: 
- Work Context — Freedom to Make Decisions (https://www.onetonline.org/find/descriptor/result/4.C.3.a.4)
    - No freedom 0 ~ 100 A lot of freedom
- Work Context — Structured versus Unstructured Work (https://www.onetonline.org/find/descriptor/result/4.C.3.b.8)
    - Structured (no freedom) 0 ~ Unstructured (a lot of freedom)
- Work Context — Time Pressure (https://www.onetonline.org/find/descriptor/result/4.C.3.d.1)
    - Never 0 ~ 100 Every day
- Work Context — Regular Work Schedules (https://www.onetonline.org/find/descriptor/result/4.C.3.d.4)
    - Regular/established schedule 0 ~ 100 Seasonal/only during certain times of the year
- Work Styles — Independence (https://www.onetonline.org/find/descriptor/result/1.C.6)
    - No independence 0 ~ 100 A lot of independence
    - "Job requires developing one's own ways of doing things, guiding oneself with little or no supervision, and depending on oneself to get things done."

These were determined to impact flexibility from the browse tool (https://www.onetonline.org/find/descriptor/browse/)

In [31]:
free = pd.read_csv('../Freedom_to_Make_Decisions.csv')
struct = pd.read_csv('../Structured_versus_Unstructured_Work.csv')
time = pd.read_csv('../Time_Pressure.csv')
sched = pd.read_csv('../Work_Schedules.csv')
indep = free = pd.read_csv('../Freedom_to_Make_Decisions.csv')

In [32]:
free = free.rename(columns = {'Context':'Freedom_to_Make_Decisions'})
struct = struct.rename(columns = {'Context':'Structured_v_Unstructured'})
time = time.rename(columns = {'Context':'Time_Pressure'})
sched = sched.rename(columns = {'Context':'Regular_Schedule'})
indep = indep.rename(columns = {'Context':'Independence'})

In [33]:
tmp = pd.merge(free, struct, how='left', on=['Code','Occupation'])
tmp = pd.merge(tmp, time, how='left', on=['Code','Occupation'])
tmp = pd.merge(tmp, sched, how='left', on=['Code','Occupation'])
tmp = pd.merge(tmp, indep, how='left', on=['Code','Occupation'])

onet = tmp
onet.rename(columns = {'Code': '2018 SOC Code'}, inplace=True) # for data straight from ONET
onet.head()

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence
0,100,29-1213.00,Dermatologists,94,70,0,100
1,100,29-1215.00,Family Medicine Physicians,87,93,0,100
2,100,23-1023.00,"Judges, Magistrate Judges, and Magistrates",98,93,0,100
3,100,29-1024.00,Prosthodontists,93,77,6,100
4,100,29-1229.06,Sports Medicine Physicians,94,72,27,100


In [34]:
onet[['2018 SOC Code','drop']] = onet['2018 SOC Code'].str.split(".", expand=True)
onet.drop(columns='drop', inplace=True)

onet.head()

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence
0,100,29-1213,Dermatologists,94,70,0,100
1,100,29-1215,Family Medicine Physicians,87,93,0,100
2,100,23-1023,"Judges, Magistrate Judges, and Magistrates",98,93,0,100
3,100,29-1024,Prosthodontists,93,77,6,100
4,100,29-1229,Sports Medicine Physicians,94,72,27,100


Trying to remove the 2010 Census code from the list in the DateFrame showed that there were missing values in the Census Code column. Investigation found that there were two values not mapping in the dictionary. 

- 2018 SOC Code 53-6031 is listed as 56-6031 in the Crosswalk file, but I am confident that this is a typo as it is listed under 53-6030. The corresponding 2010 Census code is 9360.
- 2018 SOC Code 17-3012 is listed as having a space in the Crosswalk file that was not removed in the string split procedure above. The corresponding 2010 Census code is 1540.


In [35]:
onet['2010 Census Code'] = onet['2018 SOC Code'].map(crosswalk_dict)
nulls = onet[onet['2010 Census Code'].isna()]

nulls

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence,2010 Census Code
235,85,53-6031,Automotive and Watercraft Service Attendants,69,90,11,85,NaN
667,70,17-3012,Electrical and Electronics Drafters,70,83,15,70,NaN


In [ ]:
CORRECTION1 = (onet['2018 SOC Code'] == '53-6031')
CORRECTION2 = (onet['2018 SOC Code'] == '17-3012')

onet.loc[CORRECTION1, '2010 Census Code'] = '[9360]'
onet.loc[CORRECTION2, '2010 Census Code'] = '[1540]'

In [23]:
onet.head()

,2018 SOC Code,title,teleworkable
0,11-1011,Chief Executives,1
1,11-1011,Chief Sustainability Officers,1
2,11-1021,General and Operations Managers,1
3,11-2011,Advertising and Promotions Managers,1
4,11-2021,Marketing Managers,1


Having corrected for the missing values, we can remove the code from the list. 

In [24]:
onet['2010 Census Code'] = onet['2010 Census Code'].apply(lambda x: x[0])

KeyError: '2010 Census Code'

In [ ]:
onet.head()

In [ ]:
onet.to_csv('onet_flex.csv', index=False)

## Dingel and Neiman 2020

The data generated by Dingel and Neiman 2020 uses the ONET Release 24.2, which appears to  utilize 2019 codes that must be crosswalked to the 2018 SOC codes prior to merging back to 2010 Census codes. 

In [59]:
onet = pd.read_csv('../Dingel and Neiman 2020/occupations_workathome.csv')

onet.rename(columns = {'onetsoccode': '2019 SOC Code'}, inplace=True) # for Dingel and Neiman data
onet.head()

,2019 SOC Code,title,teleworkable
0,11-1011.00,Chief Executives,1
1,11-1011.03,Chief Sustainability Officers,1
2,11-1021.00,General and Operations Managers,1
3,11-2011.00,Advertising and Promotions Managers,1
4,11-2021.00,Marketing Managers,1


In [60]:
crosswalk2019 = pd.read_csv('../2019_to_SOC_Crosswalk.csv')

In [62]:
crosswalk2019.head()

,O*NET-SOC 2019 Code,O*NET-SOC 2019 Title,2018 SOC Code,2018 SOC Title
0,11-1011.00,Chief Executives,11-1011,Chief Executives
1,11-1011.03,Chief Sustainability Officers,11-1011,Chief Executives
2,11-1021.00,General and Operations Managers,11-1021,General and Operations Managers
3,11-1031.00,Legislators,11-1031,Legislators
4,11-2011.00,Advertising and Promotions Managers,11-2011,Advertising and Promotions Managers


In [69]:
crosswalk2019.rename(columns = {'O*NET-SOC 2019 Code': '2019 SOC Code'}, inplace=True)

crosswalk2019_dict = dict(crosswalk2019.groupby('2019 SOC Code')['2018 SOC Code'].apply(list))

In [71]:
crosswalk2019_dict #11-2031.00

{'11-1011.00': ['11-1011'],
 '11-1011.03': ['11-1011'],
 '11-1021.00': ['11-1021'],
 '11-1031.00': ['11-1031'],
 '11-2011.00': ['11-2011'],
 '11-2021.00': ['11-2021'],
 '11-2022.00': ['11-2022'],
 '11-2032.00': ['11-2032'],
 '11-2033.00': ['11-2033'],
 '11-3012.00': ['11-3012'],
 '11-3013.00': ['11-3013'],
 '11-3013.01': ['11-3013'],
 '11-3021.00': ['11-3021'],
 '11-3031.00': ['11-3031'],
 '11-3031.01': ['11-3031'],
 '11-3031.03': ['11-3031'],
 '11-3051.00': ['11-3051'],
 '11-3051.01': ['11-3051'],
 '11-3051.02': ['11-3051'],
 '11-3051.03': ['11-3051'],
 '11-3051.04': ['11-3051'],
 '11-3051.06': ['11-3051'],
 '11-3061.00': ['11-3061'],
 '11-3071.00': ['11-3071'],
 '11-3071.04': ['11-3071'],
 '11-3111.00': ['11-3111'],
 '11-3121.00': ['11-3121'],
 '11-3131.00': ['11-3131'],
 '11-9013.00': ['11-9013'],
 '11-9021.00': ['11-9021'],
 '11-9031.00': ['11-9031'],
 '11-9032.00': ['11-9032'],
 '11-9033.00': ['11-9033'],
 '11-9039.00': ['11-9039'],
 '11-9041.00': ['11-9041'],
 '11-9041.01': ['11-

In [70]:
onet['2018 SOC Code'] = onet['2019 SOC Code'].map(crosswalk2019_dict)
nulls = onet[onet['2018 SOC Code'].isna()]

nulls

,2019 SOC Code,title,teleworkable,2018 SOC Code
6,11-2031.00,Public Relations and Fundraising Managers,1,NaN
7,11-3011.00,Administrative Services Managers,1,NaN
10,11-3031.02,"Financial Managers, Branch or Department",1,NaN
18,11-3071.01,Transportation Managers,1,NaN
19,11-3071.02,Storage and Distribution Managers,0,NaN
...,...,...,...,...
940,53-5021.03,"Pilots, Ship",0,NaN
949,53-6051.08,Freight and Cargo Inspectors,0,NaN
954,53-7032.00,Excavating and Loading Machine and Dragline Op...,0,NaN
955,53-7033.00,"Loading Machine Operators, Underground Mining",0,NaN


In [48]:
crosswalk.head(10)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,0050,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,0050,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,0060,NaN,11-2032,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032
9,NaN,0060,NaN,11-2033,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033


In [52]:
onet[['2018 SOC Code','drop']] = onet['2018 SOC Code'].str.split(".", expand=True)
onet.drop(columns='drop', inplace=True)

onet.head()

,2018 SOC Code,title,teleworkable
0,11-1011,Chief Executives,1
1,11-1011,Chief Sustainability Officers,1
2,11-1021,General and Operations Managers,1
3,11-2011,Advertising and Promotions Managers,1
4,11-2021,Marketing Managers,1


In [53]:
onet['2010 Census Code'] = onet['2018 SOC Code'].map(crosswalk_dict)
nulls = onet[onet['2010 Census Code'].isna()]

nulls

,2018 SOC Code,title,teleworkable,2010 Census Code
6,11-2031,Public Relations and Fundraising Managers,1,NaN
7,11-3011,Administrative Services Managers,1,NaN
36,11-9061,Funeral Service Managers,0,NaN
90,13-2021,Assessors,1,NaN
91,13-2021,"Appraisers, Real Estate",1,NaN
...,...,...,...,...
934,53-4021,"Railroad Brake, Signal, and Switch Operators",0,NaN
945,53-6031,Automotive and Watercraft Service Attendants,0,NaN
954,53-7032,Excavating and Loading Machine and Dragline Op...,0,NaN
955,53-7033,"Loading Machine Operators, Underground Mining",0,NaN
